In [1]:
from lib.ekyn import *
from torch.utils.data import DataLoader,ConcatDataset,TensorDataset
from lib.datasets import EpochedDataset
from sklearn.model_selection import train_test_split
import torch
from lib.models import RegNetX
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch import nn
from torch.nn.functional import relu

def get_loss(model,dataloader,criterion,device):
    model.to(device)
    criterion.to(device)
    with torch.no_grad():
        total = 0
        for Xi,yi in tqdm(dataloader,leave=False):
            Xi,yi = Xi.to(device),yi.to(device)
            logits = model(Xi)
            loss = criterion(logits,yi)
            total += loss.item()
    return total/len(dataloader)
CONFIG = {
    'BATCH_SIZE':128,
    'WINDOW_SIZE':5000,
    'LEARNING_RATE':3e-4,
    'TEST_SIZE':.25,
    'DEVICE':'cuda',
    'DEPTHI':[1],
    'WIDTHI':[4]
}

ids = get_ekyn_ids()
X,y = load_ekyn_pt_robust_scaled(idx=ids[0],condition='Vehicle')
X_train,X_dev,y_train,y_dev = train_test_split(X,y,shuffle=True,random_state=0)
trainloader = DataLoader(TensorDataset(X_train,y_train),batch_size=CONFIG['BATCH_SIZE'],shuffle=True)
devloader = DataLoader(TensorDataset(X_dev,y_dev),batch_size=CONFIG['BATCH_SIZE'],shuffle=True)

In [13]:
class YBlock(nn.Module):
    def __init__(self,in_channels,out_channels) -> None:
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv1d(in_channels=in_channels,out_channels=out_channels,kernel_size=3,stride=1,padding='same',bias=False),
            nn.ReLU()
        )

    def forward(self,x):
        x = self.block(x)
        return x
class RegNetY(nn.Module):
    def __init__(self,depth,width,stem_kernel_size) -> None:
        super().__init__()

        self.stem = nn.Sequential(
            nn.Conv1d(in_channels=1,out_channels=width[0],kernel_size=stem_kernel_size,stride=2,padding=stem_kernel_size//2,bias=False),
            nn.MaxPool1d(kernel_size=2,stride=2),
            nn.ReLU()
        )

        self.body = nn.Sequential()
        for stage_i in range(len(width)):
            for block_i in range(depth[stage_i]):
                print(block_i)
                self.body.add_module(name=f'{stage_i}_{block_i}',module=YBlock(in_channels=width[stage_i],out_channels=width[stage_i]))

        self.classifier = nn.Sequential(
            nn.AvgPool1d(kernel_size=1250),
            nn.Flatten(start_dim=1),
            nn.Linear(in_features=width[-1],out_features=3)
        )

    def forward(self,x):
        x = x.view(-1,1,5000)
        x = self.stem(x)
        x = self.body(x)
        x = self.classifier(x)
        return x
model = RegNetY(depth=[2],width=[4],stem_kernel_size=3)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=CONFIG['LEARNING_RATE'])
model.to(CONFIG['DEVICE'])

0
1


RegNetY(
  (stem): Sequential(
    (0): Conv1d(1, 4, kernel_size=(3,), stride=(2,), padding=(1,), bias=False)
    (1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (body): Sequential(
    (0_0): YBlock(
      (block): Sequential(
        (0): Conv1d(4, 4, kernel_size=(3,), stride=(1,), padding=same, bias=False)
        (1): ReLU()
      )
    )
    (0_1): YBlock(
      (block): Sequential(
        (0): Conv1d(4, 4, kernel_size=(3,), stride=(1,), padding=same, bias=False)
        (1): ReLU()
      )
    )
  )
  (classifier): Sequential(
    (0): AvgPool1d(kernel_size=(1250,), stride=(1250,), padding=(0,))
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=4, out_features=3, bias=True)
  )
)

In [15]:
lossi = []
trainlossi = []
trainf1 = []
devlossi = []
devf1 = []
model.train()
from lib.utils import training_loop,development_loop
for i in tqdm(range(1000)):
    loss,f1 = training_loop(model=model,trainloader=trainloader,criterion=criterion,optimizer=optimizer,device=CONFIG['DEVICE'])
    trainlossi.append(loss)
    trainf1.append(f1)

    loss,f1 = development_loop(model=model,devloader=devloader,criterion=criterion,device=CONFIG['DEVICE'])
    devlossi.append(loss)
    devf1.append(f1)

    fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,4))
    ax[0].plot(trainlossi)
    ax[0].plot(devlossi)
    ax[1].plot(trainf1)
    ax[1].plot(devf1)
    plt.savefig('loss.jpg')
    plt.close()

 68%|██████▊   | 679/1000 [02:45<01:18,  4.09it/s]


In [ ]:
from lib.utils import evaluate
print(evaluate(trainloader,model,criterion,CONFIG['DEVICE']))
print(evaluate(devloader,model,criterion,CONFIG['DEVICE']))

In [ ]:
X,y = load_ekyn_pt_robust_scaled(idx=ids[0],condition='PF')
testloader = DataLoader(TensorDataset(X,y),batch_size=CONFIG['BATCH_SIZE'],shuffle=False)
print(evaluate(testloader,model,criterion,CONFIG['DEVICE']))

In [ ]:
import pandas as pd
df = pd.DataFrame()
conditions = ['Vehicle','PF']
for idx in ids:
    for condition in conditions:
        dataloader = DataLoader(EpochedDataset(idx=idx,condition=condition),batch_size=512,shuffle=False)
        report = evaluate(dataloader,model,criterion,CONFIG['DEVICE'])[1]
        df = pd.concat([df,pd.Series(report,name=idx)],axis=1)

In [ ]:
df.T.describe()

In [ ]:
import seaborn as sns
sns.pairplot(df.T.reset_index(drop=True))

In [ ]:
testloader = DataLoader(ConcatDataset([EpochedDataset(idx=idx,condition=condition) for idx in ids[1:] for condition in ['Vehicle','PF']]),batch_size=512,shuffle=True)
print(evaluate(testloader,model,criterion,CONFIG['DEVICE']))